# Goal: Build a Company latest news summary tool

**Iteration-1: Given the Country name;**

1. look for all the recent news articles summarizied in couple of sentences with source link - each hour.
2. Use Google search for grounding
3. Limit the number of sources to 5

**Iteration-2: Add the following feature;**

1. Refresh it for every hour
2. If there is a negative news, send an alert to your email.

**Iteration-3: Add features to handle volume;**

1. Add functions, Data stores

**Iteration-4: Improve the accuracy of the results, Add evaluation methods**

1. Improve accuracy to summarize the very much the needed content
2. Add evaluation - put more emphasis here
3. Deploy as a web app in Google Cloud to end users

## Set up the system

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


# Access the API - useful for accessing the model, google search
from google import genai
from google.genai import types
from IPython.display import Markdown, HTML, display


#Set the API key
from kaggle_secrets import UserSecretsClient
GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

# Initialize the client to interact with API 
client = genai.Client(api_key=GOOGLE_API_KEY)

## Access the model's capabilities

In [2]:
# And now re-run the same query with search grounding enabled.
# tool - the component in Agent is used. Here 'google search' is a tool to retrieve the content from the web search
config_with_search = types.GenerateContentConfig(
    tools=[types.Tool(google_search=types.GoogleSearch())],
)

user_input = "You are an expert in liguistics. Access this web link in single quote. Consider only this source. And find if any name called 'Abu Ali' \
is present in the latest EU sanctions list. Provide the necessary references from this list. \
Also consider the variations of this name and aliases you are aware\
Thank you. \
'https://data.europa.eu/data/datasets/consolidated-list-of-persons-groups-and-entities-subject-to-eu-financial-sanctions?locale=en'"

user_input = "Tajmahal. "

#user_input = input("What would like to search?")

def query_with_grounding():
    response = client.models.generate_content(
        model='gemini-2.0-flash',
        contents=user_input,
        config=config_with_search
    )
    return response.candidates[0]


rc = query_with_grounding()
Markdown(rc.content.parts[0].text)

The Taj Mahal is a magnificent ivory-white marble mausoleum located on the southern bank of the Yamuna River in Agra, Uttar Pradesh, India. Here's a breakdown of key facts:

*   **Commissioned by:** The Mughal emperor Shah Jahan in 1631.
*   **Purpose:** To house the tomb of his favorite wife, Mumtaz Mahal, who died in 1631 while giving birth to their 14th child.
*   **Construction:** Construction started in 1632. The mausoleum was largely completed by 1643, but work on other parts of the complex continued until 1653.
*   **Cost:** Estimated at the time to be around 32 million rupees, which would be approximately 52.8 billion rupees (U.S. $827 million) in 2015.
*   **Artisans:** About 20,000 artisans were employed during construction.
*   **Architecture:** It's considered the best example of Mughal architecture, blending Indian, Persian, and Islamic styles. It's noted for its bilateral symmetry.
*   **UNESCO World Heritage Site:** Designated in 1983 as "the jewel of Muslim art in India and one of the universally admired masterpieces of the world's heritage."
*   **Popularity:** Attracts 7–8 million visitors a year.
*   **Materials:** Built with brick-in-lime mortar, veneered with red sandstone and marble, and features inlay work of precious and semi-precious stones.
*   **Complex:** The 17-hectare (42-acre) complex includes a mosque, a guest house, and formal gardens.
*   **Symbolism:** Intended as an earthly replica of Mumtaz Mahal's house in paradise.


In [3]:
from pprint import pprint

while not rc.grounding_metadata.grounding_supports or not rc.grounding_metadata.grounding_chunks:
    # If incomplete grounding data was returned, retry.
    rc = query_with_grounding()

chunks = rc.grounding_metadata.grounding_chunks
for chunk in chunks:
    print(f'{chunk.web.title}: {chunk.web.uri}')

coolkidfacts.com: https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqAI9HEyvViy2TTF7yAeWptRPMx0z3sBgwDAxSHmD2AUD6D3jveRtCgdRhnKZ84QrYyBkgBM310hP2-2FjddpBalFJClteoKpN2r1ShgCEVaDgyW8zLFxO2N8FT5MFjgiYQn_2CyQZUQo8oe15THPPBnQ
wikipedia.org: https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqAKPOHu3ypavAcFMMbcYukYm0GtPr_NsOC3k3W8MWOhOK-dLRLHPS8biJU9PkTU6OEmKXuGFgnKk44XpHVnY67m9lHMrL2WMYr_CAk2P8-r-vt--W8TCrGabJZipNpkXUyu9
tajmahal.gov.in: https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqAIaqkLn2s6DcbSoatoJxOiiOSS7McWcF96E6PUMmtYIe1VfQPtlDtSq5Qwzu6ezLCXqcbPjn0qgrzAy82biB-jyejq4LOwzzklMXf5spTs3i2SdeHLr86crZnIKh9eWONGTbb2F
tajmahalcabs.com: https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqAJXY2xYAVmZLiZIQumXle5NTgry4ldffJ8NfvxEdnjU2_qSGo923e2nDOJUgisAGAr37Ab1Xw5w4wszPBfavyHQZmPGgYpAOcJdIUXv9p2DNdHc-XMlsE3evZBipU4IyCWn77VTh5dIez5WpjrY3Dp-YboiFkfsGGcloQ==
agra.nic.in: https://vertexaisearch.cloud.google.com/grounding-api-redir

In [4]:
HTML(rc.grounding_metadata.search_entry_point.rendered_content)

In [5]:
from pprint import pprint
import json

def serialize(obj):
    if hasattr(obj, "__dict__"):
        return {k: serialize(v) for k, v in vars(obj).items()}
    elif isinstance(obj, list):
        return [serialize(item) for item in obj]
    else:
        return obj
    
supports = rc.grounding_metadata.grounding_supports
for support in supports:
    pprint(serialize(support))

{'confidence_scores': [0.607901, 0.98561597, 0.8753154],
 'grounding_chunk_indices': [0, 1, 2],
 'segment': {'end_index': 237,
             'part_index': None,
             'start_index': 174,
             'text': '*   **Commissioned by:** The Mughal emperor Shah Jahan '
                     'in 1631.'}}
{'confidence_scores': [0.88818276],
 'grounding_chunk_indices': [1],
 'segment': {'end_index': 365,
             'part_index': None,
             'start_index': 238,
             'text': '*   **Purpose:** To house the tomb of his favorite wife, '
                     'Mumtaz Mahal, who died in 1631 while giving birth to '
                     'their 14th child.'}}
{'confidence_scores': [0.7927082, 0.6976934, 0.977131, 0.99287367, 0.9444134],
 'grounding_chunk_indices': [3, 4, 5, 1, 6],
 'segment': {'end_index': 417,
             'part_index': None,
             'start_index': 366,
             'text': '*   **Construction:** Construction started in 1632.'}}
{'confidence_scores': [0.9222

In [6]:
import io

markdown_buffer = io.StringIO()

# Print the text with footnote markers.
markdown_buffer.write("Supported text:\n\n")
for support in supports:
    #pprint(support.segment.text)
    markdown_buffer.write(" * ")
    markdown_buffer.write(
        rc.content.parts[0].text[support.segment.start_index : support.segment.end_index]
    )

    for i in support.grounding_chunk_indices:
        chunk = chunks[i].web
        markdown_buffer.write(f"<sup>[{i+1}]</sup>")

    markdown_buffer.write("\n\n")


# And print the footnotes.
markdown_buffer.write("Citations:\n\n")
for i, chunk in enumerate(chunks, start=1):
    markdown_buffer.write(f"{i}. [{chunk.web.title}]({chunk.web.uri})\n")


Markdown(markdown_buffer.getvalue())

# Create a table

Supported text:

 * *   **Commissioned by:** The Mughal emperor Shah Jahan in 1631.<sup>[1]</sup><sup>[2]</sup><sup>[3]</sup>

 * *   **Purpose:** To house the tomb of his favorite wife, Mumtaz Mahal, who died in 1631 while giving birth to their 14th child.<sup>[2]</sup>

 * *   **Construction:** Construction started in 1632.<sup>[4]</sup><sup>[5]</sup><sup>[6]</sup><sup>[2]</sup><sup>[7]</sup>

 * The mausoleum was largely completed by 1643, but work on other parts of the complex continued until 1653.<sup>[5]</sup>

 * *   **Cost:** Estimated at the time to be around 32 million rupees, which would be approximately 52.8 billion rupees (U.S. $827 million) in 2015.<sup>[5]</sup><sup>[2]</sup>

 * *   **Artisans:** About 20,000 artisans were employed during construction.<sup>[1]</sup><sup>[5]</sup><sup>[2]</sup>

 * *   **Architecture:** It's considered the best example of Mughal architecture, blending Indian, Persian, and Islamic styles.<sup>[8]</sup>

 * It's noted for its bilateral symmetry.<sup>[6]</sup>

 * *   **UNESCO World Heritage Site:** Designated in 1983 as "the jewel of Muslim art in India and one of the universally admired masterpieces of the world's heritage."<sup>[2]</sup><sup>[5]</sup>

 * *   **Popularity:** Attracts 7–8 million visitors a year.
*<sup>[5]</sup>

 *   **Materials:** Built with brick-in-lime mortar, veneered with red sandstone and marble, and features inlay work of precious and semi-precious stones.
*<sup>[6]</sup>

 *   **Complex:** The 17-hectare (42-acre) complex includes a mosque, a guest house, and formal gardens.
*<sup>[5]</sup><sup>[2]</sup>

 *   **Symbolism:** Intended as an earthly replica of Mumtaz Mahal's house in paradise.
<sup>[7]</sup>

Citations:

1. [coolkidfacts.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqAI9HEyvViy2TTF7yAeWptRPMx0z3sBgwDAxSHmD2AUD6D3jveRtCgdRhnKZ84QrYyBkgBM310hP2-2FjddpBalFJClteoKpN2r1ShgCEVaDgyW8zLFxO2N8FT5MFjgiYQn_2CyQZUQo8oe15THPPBnQ)
2. [wikipedia.org](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqAKPOHu3ypavAcFMMbcYukYm0GtPr_NsOC3k3W8MWOhOK-dLRLHPS8biJU9PkTU6OEmKXuGFgnKk44XpHVnY67m9lHMrL2WMYr_CAk2P8-r-vt--W8TCrGabJZipNpkXUyu9)
3. [tajmahal.gov.in](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqAIaqkLn2s6DcbSoatoJxOiiOSS7McWcF96E6PUMmtYIe1VfQPtlDtSq5Qwzu6ezLCXqcbPjn0qgrzAy82biB-jyejq4LOwzzklMXf5spTs3i2SdeHLr86crZnIKh9eWONGTbb2F)
4. [tajmahalcabs.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqAJXY2xYAVmZLiZIQumXle5NTgry4ldffJ8NfvxEdnjU2_qSGo923e2nDOJUgisAGAr37Ab1Xw5w4wszPBfavyHQZmPGgYpAOcJdIUXv9p2DNdHc-XMlsE3evZBipU4IyCWn77VTh5dIez5WpjrY3Dp-YboiFkfsGGcloQ==)
5. [agra.nic.in](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqALPn7uE8byp5ToUadh425sxyFEq_MK9R1HHTqJludWt3ETGIvWR8sEKN5CsPhEr_mU03nAEM4W8XM64mgfzzeB_5bh5kEISN6Scmr3L1ssViNTYCb8jH8mZN4h5V7xrRDTAG0c-ZzZvUeya)
6. [unesco.org](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqAJfbJEqFZSygnrMFRBjbXziQts9e40Z1xXsmZ54_UBxnuCJ-vQfzxGYIvt-QikthmInrqeKXAHuvC4odEOa9grmgILZEl300_FPRcuIeXjdITcLQYGWPta8m7_HYxA=)
7. [wikipedia.org](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqALFcTS5XO99PIP1u1BLgD40uWkzLT0vcMjDLy4L8kO_iZ-vS6w_B2AfnRsHTPRioOtF42g77Ujn_uWwa_rDFeWApOusWxG1goiIsjNca-9ezhs-XZ3VUn3srNebl4gdeyp53a334VSvABI0KT8vxNWK2NgfX7xchoTBxzkaLx3MdhE=)
8. [britannica.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqAIZzBJktVawv90fd1ukQvCBVsoqG3Nmlil6fnbVBe3Zu5O35XUNg9KS4J1GO-FMTgQTdY3wZck1yaFNzjS6lgSBbHzyzziBzf12X_VtHu-pwxtEDtTZnGQwaVUid9DNWOdh44Oy)
